In [1]:
%matplotlib inline
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import pickle
import pandas as pd
import csv
import os
import time
import datetime
from tensorflow.contrib import learn
from nltk.tokenize import TweetTokenizer

In [2]:
def load_data_and_labels(train_data_file, test_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    train_data =  pd.read_csv(train_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    test_data = pd.read_csv(test_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    
    x_train = train_data['text'].tolist()
    y_train = train_data['label'].tolist()

    x_test = train_data['text'].tolist()
    y_test = train_data['label'].tolist()
    
    x_train = [s.strip() for s in x_train]
    x_test = [s.strip() for s in x_test]
    
    y_train_encoding = []
    for label in y_train:
        if label=='neutral':
            y_train_encoding.append([0, 1, 0])
        elif label=='positive':
            y_train_encoding.append([0, 0, 1])
        elif label=='negative':
            y_train_encoding.append([1, 0, 0])
    
    y_test_encoding = []
    for label in y_test:
        if label=='neutral':
            y_train_encoding.append([0, 1, 0])
        elif label=='positive':
            y_train_encoding.append([0, 0, 1])
        elif label=='negative':
            y_train_encoding.append([1, 0, 0])
    
    return [x_train, y_train_encoding, x_test, y_test_encoding]

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [3]:
final_embeddings = np.load('./data/embed_tweets_en_200M_200D/embedding_matrix.npy')
word_dict = {}
with open('./data/embed_tweets_en_200M_200D/vocabulary.pickle', 'rb') as myfile:
    word_dict = pickle.load(myfile)

In [4]:
#Load label data
x_train_sentence, y_train, x_test_sentence, y_test = load_data_and_labels('./data/supervised_data/en_train.tsv', './data/supervised_data/en_test.tsv')

max_document_length = max([len(x.split(" ")) for x in x_train_sentence])


In [5]:
tknzr = TweetTokenizer()
x_train = []
for tweet in x_train_sentence:
    temp = np.zeros(max_document_length, dtype=np.int).tolist()
    for index, word in enumerate(tknzr.tokenize(tweet)):
        word = word.lower()
        if word in word_dict:
            temp[index] = word_dict[word][0]
    x_train.append(temp)
x_train = np.array(x_train)

In [6]:
# put Word2Vec on 590 million English Tweets using 52 dimensions.
vocabulary_size = 1859185
embedding_size = 200  # Dimension of the embedding vector.
graph = tf.Graph()

sequence_length=x_train.shape[1]
num_classes=3

first_filter_window_size = 4
second_filter_window_size = 3
num_filters = 200

l2_reg_lambda=0.0

with graph.as_default():
    input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
    
    embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, embedding_size]),
                    trainable=False, name="W")

    embedding_placeholder = tf.placeholder(tf.float32, [vocabulary_size, embedding_size])
    embedding_init = embeddings.assign(embedding_placeholder)  # assign exist word embeddings

    embedded_chars = tf.nn.embedding_lookup(embeddings, input_x)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)

    # Keeping track of l2 regularization loss (optional)
    l2_loss = tf.constant(0.0)
    
     # Add variable initializer.
    init = tf.global_variables_initializer()
    
    
    # Create first cnn : a convolution + maxpool layer for each filter size    
    # Convolution Layer
    filter_shape = [first_filter_window_size, embedding_size, 1, num_filters]
    filter_shape = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
    
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    
    conv = tf.nn.conv2d(
        embedded_chars_expanded,
        filter_shape,
        strides=[1, 1, 1, 1],  # strides size change will cause cnn result different
        padding="VALID",
        name="conv")
    # Apply nonlinearity
    h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
    # Maxpooling over the outputs
    pooled = tf.nn.max_pool(
        h,
        ksize=[1, sequence_length - first_filter_window_size + 1, 1, 1],  
        strides=[1, 2, 2, 1],
        padding='VALID',
        name="pool")
    
#     # Create second cnn : a convolution + maxpool layer for each filter size
#     pooled_outputs = []
#     # Convolution Layer
#     filter_shape = [second_filter_window_size, embedding_size, 1, num_filters]
#     filter_shape = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
#     b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
#     conv = tf.nn.conv2d(
#         self.embedded_chars_expanded,
#         filter_shape,
#         strides=[1, 1, 1, 1],  # strides size change will cause cnn result different
#         padding="VALID",
#         name="conv")
#     # Apply nonlinearity
#     h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
#     # Maxpooling over the outputs
#     pooled = tf.nn.max_pool(
#         h,
#         ksize=[1, sequence_length - second_filter_window_size + 1, 1, 1],  
#         strides=[1, 1, 1, 1],
#         padding='VALID',
#         name="pool")
#     pooled_outputs.append(pooled)
            
            
            
            
     # Combine all the pooled features
#     num_filters_total = num_filters * len(filter_sizes)
    h_pool_flat = tf.reshape(pooled, [-1, num_filters])  # flatten pooling layers

    # Add dropout
#     with tf.name_scope("dropout"):
#         self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

    # Final (unnormalized) scores and predictions
    with tf.name_scope("output"):
        W = tf.get_variable(
            "W",
            shape=[num_filters, num_classes],
            initializer=tf.contrib.layers.xavier_initializer())
        b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
        l2_loss += tf.nn.l2_loss(W)
        l2_loss += tf.nn.l2_loss(b)
#         scores = tf.nn.xw_plus_b(h_drop, W, b, name="scores")
        scores = tf.add(W, b, name="scores")
        predictions = tf.argmax(scores, 1, name="predictions")

    # Calculate mean cross-entropy loss
    with tf.name_scope("loss"):
        losses = tf.nn.softmax_cross_entropy_with_logits(logits=scores, labels=input_y)
        loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

    # Accuracy
    with tf.name_scope("accuracy"):
        correct_predictions = tf.equal(predictions, tf.argmax(input_y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    


In [7]:
batch_size = 32
num_checkpoints = 5
num_epochs = 200
evaluate_every = 100
allow_soft_placement=True
log_device_placement=False

with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        
#         # Define Training procedure
#         global_step = tf.Variable(0, name="global_step", trainable=False)
#         optimizer = tf.train.AdamOptimizer(1e-3)
#         grads_and_vars = optimizer.compute_gradients(loss)
#         train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

#         # Keep track of gradient values and sparsity (optional)
#         grad_summaries = []
#         for g, v in grads_and_vars:
#             if g is not None:
#                 grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
#                 sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
#                 grad_summaries.append(grad_hist_summary)
#                 grad_summaries.append(sparsity_summary)
#         grad_summaries_merged = tf.summary.merge(grad_summaries)

#         # Output directory for models and summaries
#         timestamp = str(int(time.time()))
#         out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
#         print("Writing to {}\n".format(out_dir))

#         # Summaries for loss and accuracy
#         loss_summary = tf.summary.scalar("loss", loss)
#         acc_summary = tf.summary.scalar("accuracy", accuracy)

#         # Train Summaries
#         train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
#         train_summary_dir = os.path.join(out_dir, "summaries", "train")
#         train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

#         # Dev summaries
#         dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
#         dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
#         dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

#         # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
#         checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
#         checkpoint_prefix = os.path.join(checkpoint_dir, "model")
#         if not os.path.exists(checkpoint_dir):
#             os.makedirs(checkpoint_dir)
#         saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

#         # Write vocabulary
#         vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())


        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
#             _, step, summaries, loss, accuracy = sess.run(
#                 [train_op, global_step, train_summary_op, loss, accuracy],
#                 feed_dict)
#             time_str = datetime.datetime.now().isoformat()
#             print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
#             train_summary_writer.add_summary(summaries, step)

            loss_log, accuracy_log = sess.run([loss, accuracy], feed_dict)
            print(loss_log, accuracy_log)

#         def dev_step(x_batch, y_batch, writer=None):
#             """
#             Evaluates model on a dev set
#             """
#             feed_dict = {
#               input_x: x_batch,
#               input_y: y_batch,
#             }
#             step, summaries, loss, accuracy = sess.run(
#                 [global_step, dev_summary_op, loss, accuracy],
#                 feed_dict)
#             time_str = datetime.datetime.now().isoformat()
#             print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
#             if writer:
#                 writer.add_summary(summaries, step)

        
        sess.run(embedding_init, feed_dict={embedding_placeholder: final_embeddings})
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            
#             current_step = tf.train.global_step(sess, global_step)
#             if current_step % evaluate_every == 0:
#                 print("\nEvaluation:")
#                 dev_step(x_dev, y_dev, writer=dev_summary_writer)
#                 print("")
#             if current_step % checkpoint_every == 0:
#                 path = saver.save(sess, checkpoint_prefix, global_step=current_step)
#                 print("Saved model checkpoint to {}\n".format(path))

InvalidArgumentError: Incompatible shapes: [200] vs. [32]
	 [[Node: accuracy/Equal = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/gpu:0"](output/predictions, accuracy/ArgMax)]]
	 [[Node: accuracy/accuracy/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_87_accuracy/accuracy", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'accuracy/Equal', defined at:
  File "/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/phejimlin/anaco